In [1]:
import pickle as pkl
import pandas as pd
import os
import numpy as np
from collections import Counter
import json

with open('config.json', 'r') as f:
    config = json.load(f)
cwd = os.getcwd()
os.chdir(config['REPODIR'])
import Utils as U
from Corpus import Corpus
os.chdir(cwd)

/Users/stephentoner/miniconda3/envs/si699proj/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Load both dataset and the set difference of previous selected works, then 

In [2]:
# os.chdir('..')

In [39]:
with open('sentence_embed.pkl', 'rb') as f:
    embed = pkl.load(f)

In [40]:
embed[0]

{0: 0,
 1: 'PG105_2371',
 2: array([-0.40695864,  0.3818504 ,  0.06017515, -0.16981234, -0.4007618 ,
         0.11282735,  0.20996326,  0.1332863 , -0.47784284,  0.15134099,
         0.769952  , -0.55796057, -0.45125282,  0.4240847 , -0.07669043,
        -0.22607827, -0.05515893,  0.04954465, -0.02227528,  0.14507678,
         0.5585117 ,  0.09065207,  0.05133671,  0.48927018,  0.11336792,
         0.6518458 ,  0.30259612,  0.0792235 ,  0.14775066, -0.2883914 ,
         0.51968956,  0.7405111 ,  0.04476079,  0.29670882,  0.40674967,
         0.10144113,  0.5113817 ,  0.14497335,  0.15350078,  0.38801146,
         0.33397287, -0.5299433 , -0.14523447, -0.51450056, -0.17415676,
         0.19639616, -0.7433179 ,  0.0463079 ,  0.17458577, -0.2549399 ,
         0.05823736,  0.21992399,  0.48092613, -0.08494464, -0.51966107,
        -1.7010705 , -0.12133462,  0.35682955,  1.2218652 ,  0.29445958,
        -0.45314023, -0.14067784, -0.5998871 , -0.4438938 ,  0.7348672 ,
         0.53659564,  0

In [41]:
for e in embed:
    e['joined'] = [(e[0], e[2])]

In [42]:
df = pd.DataFrame(embed)
embed_df = df.rename(columns = {0: 'seqid', 1: 'passage_key', 2: 'sent_embeddings'})

In [43]:
embed_df = embed_df.dropna()

In [44]:
len(embed_df)

3142609

In [45]:
embed_df.columns

Index(['seqid', 'passage_key', 'sent_embeddings', 'joined'], dtype='object')

In [46]:
import itertools

In [48]:
t = embed_df.groupby('passage_key').agg({'joined':sum})

In [69]:
t['sorted'] = t.joined.apply(sorted, {'key':lambda x: x[0]})

Add Padding

In [82]:
t['raw_embedding'] = t.sorted.apply(lambda x: [_x[1] for _x in x])

In [83]:
seq_len = 50

In [84]:
t['sent_embedding'] = t.raw_embedding.apply(lambda y: y + [np.zeros(100)] * (seq_len -len(y)))

In [90]:
t= t.reset_index()

In [91]:
t_dict = t.to_dict('records')

In [92]:
t_dict[0]

{'passage_key': 'PG10008_1795',
 'joined': [(1369201,
   array([-0.2937374 ,  0.2575396 ,  0.10194786, -0.22338863, -0.17764194,
           0.14115842,  0.22782029,  0.11798462, -0.3768505 ,  0.1848621 ,
           0.57274127, -0.44233087, -0.4008299 ,  0.2706552 ,  0.01428122,
          -0.23554105, -0.01237826,  0.0258179 , -0.12332068,  0.22874717,
           0.6415113 ,  0.14470795,  0.07807157,  0.4079683 ,  0.07189687,
           0.64091885,  0.13701695,  0.04386079,  0.1421421 , -0.20019798,
           0.27662027,  0.7072582 ,  0.10105017,  0.21411698,  0.37201968,
           0.21189368,  0.37315667,  0.3200473 ,  0.3031547 ,  0.2140777 ,
           0.17831321, -0.41804594, -0.02633602, -0.5123078 , -0.17367801,
           0.10329646, -0.47458157, -0.02685583,  0.09504209, -0.33447453,
           0.00255084,  0.03761066,  0.35539317,  0.13571478, -0.47088233,
          -2.085241  , -0.08156738,  0.21013084,  1.3416381 ,  0.39734292,
          -0.3385919 ,  0.01824415, -0.4894650

In [93]:
with open ('embedding_data_final.pkl', 'wb') as f:
    pkl.dump(t_dict, f)

In [74]:
max(len(x) for x in t.sorted)

50

Split data into sentence embeddings and author id

In [68]:
len(embed_df.joined.iloc[0])

1

In [13]:
pd.DataFrame(X_train, y_train)

In [15]:
seed_works[seed_works['id'] == 'PG53940']

,Unnamed: 0,id,title,author,authoryearofbirth,authoryearofdeath,language,downloads,subjects,type,subject_set,Sub_A,Sub_B,Sub_C
7975,53938,PG53940,"Haw-Ho-Noo; Or, Records of a Tourist","Lanman, Charles",1819.0,1895.0,['en'],3.0,"{'Fishing -- North America', 'United States --...",Text,"['Description and travel', 'North America', 'F...",Description and travel,North America,Fishing


In [19]:
for_merge = seed_works[['id', 'authoryearofbirth', 'authoryearofdeath', 'downloads', 'subject_set', 'Sub_A', 'Sub_B', 'Sub_C']]

In [25]:
ff_df_null = ff_df[ff_df.authoryearofbirth.isna()]
ff_df = ff_df[~ff_df.authoryearofbirth.isna()]

ff_df_null = ff_df_null[['author_id', 'author_name', 'book_id', 'gutenbergbookid', 'title',
       'text', 'text_lines']]
null_values = ff_df_null.merge(for_merge, left_on='book_id', right_on = 'id')
null_values['topic'] = null_values['Sub_A']

In [39]:
ff_df = pd.concat((ff_df, ff_df_null))

Unique works for each of the three datasets:

In [40]:

len(local_df.book_id.unique())
# len(final_df.book_id.unique())
len(ff_df.book_id.unique())


6424

Because the final_df has a lot of text from non-English languages, I have decided to proceed as follows:
* Our overall dataset will be the `ff_df` object, corresponding to the `data_vFF.pkl` file. The filtering for this file was the most stringent.
* We will use the works not included in `ff_df`, but in `local_df`, as the training corpus for our word embeddings

In [42]:
local_df['str_text_lines'] = local_df['text_lines'].apply(str)
local_df['passage_key'] = local_df['gutenbergbookid'] + '_' + local_df['str_text_lines']

ff_df['str_text_lines'] = ff_df['text_lines'].apply(str)
ff_df['passage_key'] = ff_df['gutenbergbookid'] + '_' + ff_df['str_text_lines']


In [43]:
dataset_passages = list(ff_df.passage_key.unique())

In [44]:
ldf_index = local_df.set_index('passage_key').index
ff_index = ff_df.set_index('passage_key').index
corpus_index = ldf_index.difference(ff_index)

In [45]:
local_df_idx = local_df.set_index('passage_key')
corpus_df = local_df_idx.loc[corpus_index].reset_index()

In [46]:
corpus_df['joined_text'] = corpus_df.text.apply(' '.join)

In [48]:
corpus_text = " ".join(corpus_df.joined_text)

In [49]:
with open('corpus_text.txt', 'w') as f:
    f.write(corpus_text)

In [57]:
os.chdir(config['REPODIR'])

In [56]:
ff_out = ff_df.to_dict(orient='records')
U.write_file(ff_out, 'data_vFF.pkl', 'pkl', config['DATADIR'])

TypeError: file must have a 'write' attribute